In [12]:
import os
import re
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import nltk
from nltk.corpus import stopwords
from keras import initializers, regularizers, constraints
from keras.preprocessing import sequence
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.models import Sequential, load_model, Model
from keras.layers import (
    Dense,
    Dropout,
    Activation,
    MaxPooling1D,
    Flatten,
    Embedding,
    Conv1D,
    GlobalMaxPooling1D,
    BatchNormalization,
    CuDNNGRU,
    LSTM,
    Bidirectional,
    Input,
    concatenate,
)
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.layers import Layer
from tensorflow.keras.optimizers import Nadam
from keras.utils.vis_utils import plot_model
from keras import backend as K
from numpy import random
import gensim
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations


In [13]:
df = pd.read_csv('reviewContent+metadataBalanced.csv')

In [14]:
df=df.dropna() 

In [15]:
df

,user_id,prod_id,rating,label,date,text_
0,5044,0,1,0,11/16/2014,"Drinks were bad, the hot chocolate was watered..."
1,5045,0,1,0,9/8/2014,This was the worst experience I've ever had a ...
2,5046,0,3,0,10/6/2013,This is located on the site of the old Spruce ...
3,5047,0,5,0,11/30/2014,I enjoyed coffee and breakfast twice at Toast ...
4,5048,0,5,0,8/28/2014,I love Toast! The food choices are fantastic -...
...,...,...,...,...,...,...
160927,265253,5041,5,0,11/15/2011,Great local lounge spot! After work and regula...
160928,265316,5042,5,0,12/24/2014,I have been to this place for the third time i...
160929,62901,5043,1,0,8/4/2011,Horrible service. I saw a customer accidentall...
160930,201246,5043,2,0,4/14/2013,Its your standard late-night drunk filled coll...


In [16]:
df.describe()


,user_id,prod_id,rating,label
count,160932.000000,160932.000000,160932.000000,160932.000000
mean,82018.298940,1727.836900,3.834868,0.500000
std,74370.033461,1559.997545,1.289868,0.500002
min,5044.000000,0.000000,1.000000,0.000000
25%,23998.750000,400.000000,3.000000,0.000000
50%,51973.500000,963.000000,4.000000,0.500000
75%,131655.250000,3122.000000,5.000000,1.000000
max,265319.000000,5043.000000,5.000000,1.000000


In [17]:
len(df['prod_id'].unique())

4456

In [18]:

# Function to calculate Maximum Number of Reviews (MNR)
def calculate_mnr(df1):
    df1['date'] = pd.to_datetime(df1['date'])
    df1['reviews_per_day'] = df1.groupby(['user_id', df1['date'].dt.date])['user_id'].transform('count')
    max_reviews_per_day = df1.groupby('user_id')['reviews_per_day'].max()
    return max_reviews_per_day

# Function to calculate Percentage of Positive Reviews (PR)
def calculate_pr(df1):
    # Group reviews by user_id and create a list of reviews for each user
    user_reviews = df1.groupby('user_id')['rating'].apply(list).reset_index(name='reviews')

    # Calculate the percentage of positive reviews for each user
    user_reviews['pr'] = user_reviews['reviews'].apply(
        lambda reviews: (
            sum(rating in [4, 5] for rating in reviews) / len(reviews)
        ) if len(reviews) > 0 else 0
    )

    # Merge the result back to the original dataframe
    pr_df = user_reviews[['user_id', 'pr']]
    return pr_df

# Function to calculate Review Length (RL)
def calculate_rl(df1):
    df1['review_length'] = df1['text_'].apply(lambda x: len(x.split()))
    avg_review_length = df1.groupby('user_id')['review_length'].mean()
    return avg_review_length

# Function to calculate Reviewer Deviation (RD)
def calculate_rd(df1):
    average_ratings_per_product = df1.groupby('prod_id')['rating'].mean()
    tempdf = pd.merge(df1, average_ratings_per_product, on='prod_id', how='left')
    tempdf['rating_deviation'] = np.abs(tempdf['rating_x'] - tempdf['rating_y'])
    average_deviation_per_user = tempdf.groupby('user_id')['rating_deviation'].mean()
    return average_deviation_per_user

# Function to calculate Maximum Content Similarity (MCS)
def calculate_mcs(df1):
    # Group by user_id and collect all reviews
    user_reviews = df1.groupby('user_id')['text_'].agg(lambda x: list(map(str, x))).reset_index()

    # Initialize lists to store user_ids and max similarities
    user_ids = []
    max_similarities = []
    user_reviews_dict = dict(zip(user_reviews['user_id'], user_reviews['text_']))



    # Create a TF-IDF vectorizer outside the loop
    vectorizer = TfidfVectorizer(stop_words='english')

    for user_id, reviews in user_reviews_dict.items():
        # If there is only one review, similarity is 0
        if len(reviews) <= 1:
            user_ids.append(user_id)
            max_similarities.append(0)
        else:
            reviews = list(map(str, reviews))

            # Calculate TF-IDF vectors and cosine similarity between all pairs of reviews
            tfidf_matrix = vectorizer.fit_transform(reviews)
            similarity_matrix = cosine_similarity(tfidf_matrix)

            # Set the diagonal elements to 0 to avoid self-comparison
            np.fill_diagonal(similarity_matrix, 0)

            # Find the maximum similarity value
            max_similarity = np.max(similarity_matrix)
            
            # Append user_id and corresponding maximum similarity
            user_ids.append(user_id)
            max_similarities.append(max_similarity)

    return pd.DataFrame({'user_id': user_ids, 'mcs': max_similarities})

In [19]:
copy_df=df.copy()

In [20]:
df

,user_id,prod_id,rating,label,date,text_
0,5044,0,1,0,11/16/2014,"Drinks were bad, the hot chocolate was watered..."
1,5045,0,1,0,9/8/2014,This was the worst experience I've ever had a ...
2,5046,0,3,0,10/6/2013,This is located on the site of the old Spruce ...
3,5047,0,5,0,11/30/2014,I enjoyed coffee and breakfast twice at Toast ...
4,5048,0,5,0,8/28/2014,I love Toast! The food choices are fantastic -...
...,...,...,...,...,...,...
160927,265253,5041,5,0,11/15/2011,Great local lounge spot! After work and regula...
160928,265316,5042,5,0,12/24/2014,I have been to this place for the third time i...
160929,62901,5043,1,0,8/4/2011,Horrible service. I saw a customer accidentall...
160930,201246,5043,2,0,4/14/2013,Its your standard late-night drunk filled coll...


In [21]:

print("Calculating MNR...")
mnr_df = calculate_mnr(copy_df)
print("MNR calculation completed.")

print("Calculating PR...")
pr_df = calculate_pr(copy_df)
print("PR calculation completed.")

print("Calculating RL...")
rl_df = calculate_rl(copy_df)
print("RL calculation completed.")

print("Calculating RD...")
rd_df = calculate_rd(copy_df)
print("RD calculation completed.")

print("Calculating MCS...")
mcs_df = calculate_mcs(copy_df)
print("MCS calculation completed.")



Calculating MNR...
MNR calculation completed.
Calculating PR...
PR calculation completed.
Calculating RL...
RL calculation completed.
Calculating RD...
RD calculation completed.
Calculating MCS...
MCS calculation completed.


In [22]:
df

,user_id,prod_id,rating,label,date,text_
0,5044,0,1,0,11/16/2014,"Drinks were bad, the hot chocolate was watered..."
1,5045,0,1,0,9/8/2014,This was the worst experience I've ever had a ...
2,5046,0,3,0,10/6/2013,This is located on the site of the old Spruce ...
3,5047,0,5,0,11/30/2014,I enjoyed coffee and breakfast twice at Toast ...
4,5048,0,5,0,8/28/2014,I love Toast! The food choices are fantastic -...
...,...,...,...,...,...,...
160927,265253,5041,5,0,11/15/2011,Great local lounge spot! After work and regula...
160928,265316,5042,5,0,12/24/2014,I have been to this place for the third time i...
160929,62901,5043,1,0,8/4/2011,Horrible service. I saw a customer accidentall...
160930,201246,5043,2,0,4/14/2013,Its your standard late-night drunk filled coll...


In [23]:

# Merge the calculated features with the original dataset on 'user_id'
df = df.merge(mnr_df, on='user_id', how='left')
df = df.merge(pr_df, on='user_id', how='left')
df = df.merge(rl_df, on='user_id', how='left')
df = df.merge(rd_df, on='user_id', how='left')
df = df.merge(mcs_df, on='user_id', how='left')

In [25]:
df = df.rename(columns={'reviews_per_day': 'maximum_review_per_day_for_user', 'pr': 'percent_postive_reviews_by_user','review_length': 'max_user_review_length','rating_deviation': 'avg_user_rating_deviation_per_product','mcs': 'max_cosine_similarity_to_user_reviews'})

In [26]:
df

,user_id,prod_id,rating,label,date,text_,maximum_review_per_day_for_user,percent_postive_reviews_by_user,max_user_review_length,avg_user_rating_deviation_per_product,max_cosine_similarity_to_user_reviews
0,5044,0,1,0,11/16/2014,"Drinks were bad, the hot chocolate was watered...",1,0.000000,36.000000,2.613636,0.000000
1,5045,0,1,0,9/8/2014,This was the worst experience I've ever had a ...,1,0.000000,248.000000,2.613636,0.000000
2,5046,0,3,0,10/6/2013,This is located on the site of the old Spruce ...,1,0.250000,45.250000,0.723817,0.100917
3,5047,0,5,0,11/30/2014,I enjoyed coffee and breakfast twice at Toast ...,1,1.000000,233.000000,1.386364,0.000000
4,5048,0,5,0,8/28/2014,I love Toast! The food choices are fantastic -...,1,1.000000,152.000000,1.386364,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
160927,265253,5041,5,0,11/15/2011,Great local lounge spot! After work and regula...,1,1.000000,28.000000,0.303030,0.000000
160928,265316,5042,5,0,12/24/2014,I have been to this place for the third time i...,1,1.000000,110.000000,0.000000,0.000000
160929,62901,5043,1,0,8/4/2011,Horrible service. I saw a customer accidentall...,11,0.916667,25.416667,1.133310,0.197985
160930,201246,5043,2,0,4/14/2013,Its your standard late-night drunk filled coll...,1,0.000000,69.000000,0.333333,0.000000


In [27]:
df_no_meta = df.drop(["user_id", "prod_id","date"], axis=1)


In [28]:
df_no_meta1 =df_no_meta.sample(frac=1,random_state=1)

In [29]:
df_no_meta1.head()

,rating,label,text_,maximum_review_per_day_for_user,percent_postive_reviews_by_user,max_user_review_length,avg_user_rating_deviation_per_product,max_cosine_similarity_to_user_reviews
96371,5,0,Best Italian shop in the area. I am a NYC tran...,2,0.500000,37.000000,1.540780,0.062032
103402,4,0,Have been going here for years. Food is always...,1,1.000000,35.000000,0.138614,0.000000
160285,1,0,I was expecting more from this place. I only e...,1,0.000000,66.000000,1.863636,0.000000
99317,4,0,I got take out food here recently and was plea...,1,1.000000,59.000000,0.500000,0.000000
132540,3,1,I've been going here since high school and to ...,1,0.333333,104.333333,0.962376,0.046731


In [30]:

label_encoder = preprocessing.LabelEncoder() 

df_no_meta1['label']= label_encoder.fit_transform(df_no_meta1['label']) 

df_no_meta1['label'].unique() 

array([0, 1], dtype=int64)

In [31]:
df_no_meta1

,rating,label,text_,maximum_review_per_day_for_user,percent_postive_reviews_by_user,max_user_review_length,avg_user_rating_deviation_per_product,max_cosine_similarity_to_user_reviews
96371,5,0,Best Italian shop in the area. I am a NYC tran...,2,0.500000,37.000000,1.540780,0.062032
103402,4,0,Have been going here for years. Food is always...,1,1.000000,35.000000,0.138614,0.000000
160285,1,0,I was expecting more from this place. I only e...,1,0.000000,66.000000,1.863636,0.000000
99317,4,0,I got take out food here recently and was plea...,1,1.000000,59.000000,0.500000,0.000000
132540,3,1,I've been going here since high school and to ...,1,0.333333,104.333333,0.962376,0.046731
...,...,...,...,...,...,...,...,...
73349,5,1,The food is so fresh! Love it!,1,1.000000,7.000000,0.924862,0.000000
109259,4,0,More than a sports bar. Good food and service.,3,0.750000,15.500000,0.851449,0.213593
50057,4,1,One of those restaurants where you truly feel ...,1,1.000000,45.500000,0.486437,0.000000
5192,4,1,This place is cute! I went here long time ago....,1,0.500000,65.000000,0.825269,0.038469


In [32]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x

In [33]:
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [34]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_data(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    return text
    
df_no_meta1['text_'] = df_no_meta1['text_'].apply(clean_data)

c:\Users\prems\anaconda3\envs\swm\lib\site-packages\ipykernel_launcher.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  # This is added back by InteractiveShellApp.init_path()


In [35]:
df_no_meta1

,rating,label,text_,maximum_review_per_day_for_user,percent_postive_reviews_by_user,max_user_review_length,avg_user_rating_deviation_per_product,max_cosine_similarity_to_user_reviews
96371,5,0,best italian shop in the area i am a nyc trans...,2,0.500000,37.000000,1.540780,0.062032
103402,4,0,have been going here for years food is always ...,1,1.000000,35.000000,0.138614,0.000000
160285,1,0,i was expecting more from this place i only ex...,1,0.000000,66.000000,1.863636,0.000000
99317,4,0,i got take out food here recently and was plea...,1,1.000000,59.000000,0.500000,0.000000
132540,3,1,i have been going here since high school and t...,1,0.333333,104.333333,0.962376,0.046731
...,...,...,...,...,...,...,...,...
73349,5,1,the food is so fresh ! love it !,1,1.000000,7.000000,0.924862,0.000000
109259,4,0,more than a sports bar good food and service,3,0.750000,15.500000,0.851449,0.213593
50057,4,1,one of those restaurants where you truly feel ...,1,1.000000,45.500000,0.486437,0.000000
5192,4,1,this place is cute ! i went here long time ago...,1,0.500000,65.000000,0.825269,0.038469


In [16]:
# X = X.map(lambda a: clean_data(a))


In [36]:
X_text = df_no_meta1.text_
X_numeric = df_no_meta1[['maximum_review_per_day_for_user','percent_postive_reviews_by_user','max_user_review_length','avg_user_rating_deviation_per_product','max_cosine_similarity_to_user_reviews']]
y = df_no_meta1.label

In [37]:
x_train_text, x_test_text, x_train_numeric, x_test_numeric, y_train, y_test = train_test_split(
    X_text, X_numeric, y, stratify=y, test_size=0.2, random_state=1
)

In [38]:
tokenizer = Tokenizer(num_words=None,lower=True,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',split=' ',char_level=False)

In [39]:
tokenizer.fit_on_texts(X_text)


In [40]:
x_train_text = tokenizer.texts_to_sequences(x_train_text)

In [41]:
x_test_text = tokenizer.texts_to_sequences(x_test_text)

In [42]:
word_index = tokenizer.word_index

In [43]:
X_text = tokenizer.texts_to_sequences(X_text)

In [44]:
vocab_size = len(word_index)
print('Vocab size: {}'.format(vocab_size))
longest = max(len(seq) for seq in X_text)
print("Longest comment size: {}".format(longest))
average = np.mean([len(seq) for seq in X_text])
print("Average comment size: {}".format(average))
stdev = np.std([len(seq) for seq in X_text])
print("Stdev of comment size: {}".format(stdev))
max_len = int(average + stdev * 3)
print('Max comment size: {}'.format(max_len))

Vocab size: 87775
Longest comment size: 995
Average comment size: 105.61196033107151
Stdev of comment size: 102.15050913314423
Max comment size: 412


In [45]:
processed_post_x_train = pad_sequences(x_train_text, maxlen=max_len, padding='post', truncating='post')
processed_post_x_test = pad_sequences(x_test_text, maxlen=max_len, padding='post', truncating='post')

In [46]:
processed_x_train = pad_sequences(x_train_text, maxlen=max_len)
processed_x_test = pad_sequences(x_test_text, maxlen=max_len)

In [47]:
processed_pre_x_train = pad_sequences(x_train_text, maxlen=max_len)
processed_pre_x_test = pad_sequences(x_test_text, maxlen=max_len)

In [48]:
print('x_train shape:', processed_x_train.shape)
print('x_test shape:', processed_x_test.shape)


x_train shape: (120699, 412)
x_test shape: (40233, 412)


In [49]:
embeddings_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [50]:
embedding_dim = 100
k = 0
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        k += 1
        embedding_matrix[i] = embedding_vector

In [51]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        #self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    #print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim

In [68]:
def get_model():
    review_input = Input(shape=(max_len,), dtype='int32')
    review_input_post = Input(shape=(max_len,), dtype='int32')
    mnr_input = Input(shape=(1,), dtype='float32')  # Add MNR as a numeric input
    pr_input = Input(shape=(1,), dtype='float32')   # Add PR as a numeric input
    rl_input = Input(shape=(1,), dtype='float32')   # Add RL as a numeric input
    rd_input = Input(shape=(1,), dtype='float32')   # Add RD as a numeric input
    mcs_input = Input(shape=(1,), dtype='float32')  # Add MCS as a numeric input

    x1 = Embedding(vocab_size + 1, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True)(review_input)
    x1 = Bidirectional(LSTM(60, return_sequences=True))(x1)
    x1 = Dropout(0.3)(x1)
    x1 = Attention(max_len)(x1)

    x2 = Embedding(vocab_size + 1, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True)(review_input_post)
    x2 = Bidirectional(LSTM(60, return_sequences=True))(x2)
    x2 = Dropout(0.3)(x2)
    x2 = Attention(max_len)(x2)

    x = concatenate([x1, x2, mnr_input, pr_input, rl_input, rd_input, mcs_input])
    x = Dense(50, activation='relu')(x)
    x= Dropout(0.2)(x)
    x = BatchNormalization()(x)
    preds = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=[review_input, review_input_post, mnr_input, pr_input, rl_input, rd_input, mcs_input], outputs=preds)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [69]:
K._get_available_gpus()

['/device:GPU:0']

In [70]:
model = get_model()


In [71]:
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 412)]        0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 412)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 412, 100)     8777600     input_8[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 412, 100)     8777600     input_9[0][0]                    
____________________________________________________________________________________________

In [72]:
from keras.callbacks import ModelCheckpoint


In [73]:
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [74]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=10)

In [75]:
x_train_numeric


,maximum_review_per_day_for_user,percent_postive_reviews_by_user,max_user_review_length,avg_user_rating_deviation_per_product,max_cosine_similarity_to_user_reviews
1889,1,0.0,14.0,0.505807,0.000000
7516,1,1.0,38.0,0.687131,0.000000
148778,1,1.0,65.0,0.769231,0.000000
88431,1,1.0,154.0,1.000000,0.000000
65035,2,0.5,31.0,0.363070,0.000000
...,...,...,...,...,...
123085,1,1.0,58.0,0.842105,0.000000
12978,1,1.0,93.0,1.112426,0.000000
111290,1,1.0,21.0,1.208904,0.000000
151097,1,0.0,34.0,1.714286,0.000000


In [76]:
history = model.fit([processed_x_train,processed_pre_x_train,x_train_numeric['maximum_review_per_day_for_user'],x_train_numeric['percent_postive_reviews_by_user'],x_train_numeric['max_user_review_length'],x_train_numeric['avg_user_rating_deviation_per_product'],x_train_numeric['max_cosine_similarity_to_user_reviews']],y_train, validation_data=([processed_x_test,processed_pre_x_test,x_test_numeric['maximum_review_per_day_for_user'],x_test_numeric['percent_postive_reviews_by_user'],x_test_numeric['max_user_review_length'],x_test_numeric['avg_user_rating_deviation_per_product'],x_test_numeric['max_cosine_similarity_to_user_reviews']],y_test), epochs=10,batch_size=32,callbacks=[early_stopping_monitor],verbose=1)


Epoch 1/10
3772/3772 [==============================] - 450s 117ms/step - loss: 0.5640 - accuracy: 0.7081 - val_loss: 0.5187 - val_accuracy: 0.7373
Epoch 2/10
3772/3772 [==============================] - 446s 118ms/step - loss: 0.4592 - accuracy: 0.7865 - val_loss: 0.5110 - val_accuracy: 0.7440
Epoch 3/10
3772/3772 [==============================] - 457s 121ms/step - loss: 0.3916 - accuracy: 0.8236 - val_loss: 0.5506 - val_accuracy: 0.7450
Epoch 4/10
3772/3772 [==============================] - 454s 120ms/step - loss: 0.3364 - accuracy: 0.8538 - val_loss: 0.5773 - val_accuracy: 0.7377
Epoch 5/10
3772/3772 [==============================] - 453s 120ms/step - loss: 0.2872 - accuracy: 0.8768 - val_loss: 0.6455 - val_accuracy: 0.7330
Epoch 6/10
3772/3772 [==============================] - 446s 118ms/step - loss: 0.2419 - accuracy: 0.8978 - val_loss: 0.7258 - val_accuracy: 0.7262
Epoch 7/10
3772/3772 [==============================] - 444s 118ms/step - loss: 0.2013 - accuracy: 0.9160 - val_